In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

!rm -rf ./logs/GRU

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename):
    data = np.matrix(pd.read_csv(filename).values)

    #we need to transpose data sets to be compatable with our weight vectors
    data_y = data[:, 0]
    data_y-=1 #binaryization of the categorical data

    data_x = data[:, 1:]
    data_x = convert_3D(normalize(data_x))
    return data_x, data_y

In [5]:
training_data_x, training_data_y = features_and_labels("../archive/exoTrain.csv")
test_x, test_y = features_and_labels("../archive/exoTest.csv")

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [6]:
GRUmodel = Sequential([
    GRU(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    GRU(64),
    #Dropout(0.3),
    #GRU(64),
    Dense(1, activation='sigmoid')
])

GRUmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy"]
)

log_dir = "logs/GRU/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

csv_logger = CSVLogger('training.log')

callbacks

GRUmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[csv_logger])

model = GRUmodel
model.save('keras-models/KerasGRU')

Epoch 1/10
112/112 [==============================] - 25s 193ms/step - loss: 0.6872 - accuracy: 0.9110 - val_loss: 0.6800 - val_accuracy: 0.9902
Epoch 2/10
112/112 [==============================] - 21s 188ms/step - loss: 0.6720 - accuracy: 0.9899 - val_loss: 0.6629 - val_accuracy: 0.9921
Epoch 3/10
112/112 [==============================] - 21s 192ms/step - loss: 0.6521 - accuracy: 0.9910 - val_loss: 0.6394 - val_accuracy: 0.9928
Epoch 4/10
112/112 [==============================] - 25s 221ms/step - loss: 0.6240 - accuracy: 0.9916 - val_loss: 0.6054 - val_accuracy: 0.9928
Epoch 5/10
112/112 [==============================] - 21s 188ms/step - loss: 0.5833 - accuracy: 0.9916 - val_loss: 0.5563 - val_accuracy: 0.9928
Epoch 6/10
112/112 [==============================] - 21s 186ms/step - loss: 0.5229 - accuracy: 0.9919 - val_loss: 0.4817 - val_accuracy: 0.9941
Epoch 7/10
112/112 [==============================] - 21s 187ms/step - loss: 0.4317 - accuracy: 0.9919 - val_loss: 0.3704 - val_ac

INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


In [7]:
%tensorboard --logdir logs/GRU/fit